In [2]:
# Importing the required packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
import mlflow
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
!ipykernel --version

'ipykernel' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
dataset = pd.read_csv("train.csv")

In [5]:
np.isnan(dataset.toarray())

AttributeError: 'DataFrame' object has no attribute 'toarray'

In [4]:
# mlflow.set_tracking_uri("http://192.168.0.1:5000")
# mlflow.set_tracking_uri("http://0.0.0.0:5001/")

# load the dataset
dataset = pd.read_csv("train.csv")
X = dataset.drop(columns=['Loan_ID', 'Loan_Status'])
y = dataset.Loan_Status

numerical_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

#imputing missing numerical values
imputer_num = SimpleImputer(strategy='median')
df_num_imputed = pd.DataFrame(imputer_num.fit_transform(X.select_dtypes(include=['int64','float64'])), columns=numerical_cols)

#imputing missing categarical values
imputer_cat = SimpleImputer(strategy='most_frequent')
df_cat_imputed = pd.DataFrame(imputer_cat.fit_transform(X.select_dtypes(include=['object'])), columns=categorical_cols)

cat_encoder = OneHotEncoder()
encoded = cat_encoder.fit_transform(df_cat_imputed)
df_cat_encoded = pd.DataFrame(encoded.toarray(),columns=cat_encoder.get_feature_names_out()) 
X = pd.concat([df_num_imputed,df_cat_encoded], axis=1)

In [5]:
# Take care of outliers
X[numerical_cols] = X[numerical_cols].apply(lambda x: x.clip(*x.quantile([0.05, 0.95])))
# Log Transforamtion & Domain Processing
X['LoanAmount'] = np.log(X['LoanAmount']).copy()
X['TotalIncome'] = X['ApplicantIncome'] + X['CoapplicantIncome']
X['TotalIncome'] = np.log(X['TotalIncome']).copy()

# Dropping ApplicantIncome and CoapplicantIncome
X = X.drop(columns=['ApplicantIncome','CoapplicantIncome'])


In [6]:
y.shape

(614,)

In [7]:

# Train test split
y = dataset.Loan_Status
le = LabelEncoder()
y = le.fit_transform(y)
RANDOM_SEED = 6

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.3, random_state = RANDOM_SEED)


In [8]:
# RandomForest
rf = RandomForestClassifier(random_state=RANDOM_SEED)
param_grid_forest = {
    'n_estimators': [200,400, 700],
    'max_depth': [10,20,30],
    'criterion' : ["gini", "entropy"],
    'max_leaf_nodes': [50, 100]
}

grid_forest = GridSearchCV(
        estimator=rf,
        param_grid=param_grid_forest, 
        cv=5, 
        n_jobs=-1, 
        scoring='accuracy',
        verbose=0
    )
model_forest = grid_forest.fit(X_train, y_train)

#Logistic Regression

lr = LogisticRegression(random_state=RANDOM_SEED)
param_grid_log = {
    'C': [100, 10, 1.0, 0.1, 0.01],
    'penalty': ['l1','l2'],
    'solver':['liblinear']
}

grid_log = GridSearchCV(
        estimator=lr,
        param_grid=param_grid_log, 
        cv=5,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
model_log = grid_log.fit(X_train, y_train)

#Decision Tree

dt = DecisionTreeClassifier(
    random_state=RANDOM_SEED
)

param_grid_tree = {
    "max_depth": [3, 5, 7, 9, 11, 13],
    'criterion' : ["gini", "entropy"],
}

grid_tree = GridSearchCV(
        estimator=dt,
        param_grid=param_grid_tree, 
        cv=5,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
model_tree = grid_tree.fit(X_train, y_train)

c:\Users\mrafi\anaconda3\envs\mflowenv\lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\mrafi\anaconda3\envs\mflowenv\lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
